# Google Search grounding


In [1]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display
import os

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [2]:
for model in genai.list_models():
    if "002" in model.name:
        print(model.name)

models/gemini-1.5-pro-002
models/gemini-1.5-flash-002


## Use search grounding

To enable search grounding, you specify it as a tool: `google_search_retrieval.` Like other tools, this can be supplied as a parameter to the model (to use on all chat turns or calls to generate_content), or it can be supplied per-turn to `chat.send_message.`

In [3]:
# Ask for information without search grounding.
model = genai.GenerativeModel("gemini-1.5-flash-002")

response = model.generate_content("When and where is Taylor Swift's next concert?")

Markdown(response.text)

I do not have access to real-time information, including live concert schedules.  To find the date and location of Taylor Swift's next concert, you should check these resources:

* **Taylor Swift's Official Website:** This is the most reliable source.
* **Ticketmaster:**  Check for on-sale dates and venues.
* **Other major ticket vendors:**  Sites like SeatGeek, Vivid Seats, etc., often list upcoming concerts.
* **Taylor Swift's social media:**  She may announce tour dates on her official accounts.


Keep in mind that concert schedules can change, so always check the official sources closest to the date you're interested in.


In [4]:
model = genai.GenerativeModel(
    "gemini-1.5-flash-002",
    tools="google_search_retrieval")

response = model.generate_content("When and where is Taylor Swift's next concert?")
rc = response.candidates[0]

Markdown(rc.content.parts[0].text)

Based on the provided sources, Taylor Swift's next concert is on Thursday, November 14th, 2024, at the Rogers Centre in Toronto, Canada.  She will also have a second concert at the same venue on a date not specified in the provided text.


**Response metadata**

When search grounding is used, the model returns extra metadata that includes links to search suggestions, supporting documents and information on how the supporting documents were used.

Each "grounding chunk" represents information retrieved from Google Search that was used in the grounded generation request. Following the URI will take you to the source.

In [5]:
chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(chunk)

web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfz0rSO29KuNVoSk-Wc4qmgktrz92dSH3_oa9XIoYoI1TGk_V0NfrUd4C7qoRra4E93zbJ32-7OCdkZM4_xyoJfOj1H-88DzxMGkHqsURVQNFywEEkA_1E8qHsE0rA=="
  title: "taylorswift.com"
}



As part of the response, there is a standalone styled HTML content block that you use to link back to relevant search suggestions related to the generation.

In [6]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

The `grounding_supports` in the metadata provide a way for you to correlate the grounding chunks used to the generated output text.

In [7]:
supports = rc.grounding_metadata.grounding_supports
for support in supports:
    print(support)

segment {
  end_index: 136
  text: "Based on the provided sources, Taylor Swift\'s next concert is on Thursday, November 14th, 2024, at the Rogers Centre in Toronto, Canada."
}
grounding_chunk_indices: 0
confidence_scores: 0.656059444



These supports can be used to highlight text in the response, or build tables of footnotes.

In [8]:
import io

markdown_buffer = io.StringIO()

# Print the text with footnote markers.
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        response.text[support.segment.start_index : support.segment.end_index]
    )

    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    markdown_buffer.write("\n\n")


# And print the footnotes.
markdown_buffer.write("Citations:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"* {i}: [{chunk.web.title}]({chunk.web.uri})\n")


Markdown(markdown_buffer.getvalue())

Supported text:

 * Based on the provided sources, Taylor Swift's next concert is on Thursday, November 14th, 2024, at the Rogers Centre in Toronto, Canada.<sup>[1]</sup>

Citations:

* 1: [taylorswift.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfz0rSO29KuNVoSk-Wc4qmgktrz92dSH3_oa9XIoYoI1TGk_V0NfrUd4C7qoRra4E93zbJ32-7OCdkZM4_xyoJfOj1H-88DzxMGkHqsURVQNFywEEkA_1E8qHsE0rA==)


## Dynamic grounding

In a context where you may not know in advance whether to enable search grounding or not, you can provide the model with a threshold over which it will use search grounding. This is helpful in conversational contexts, where not every turn of conversation requires search data to support a response.

If you know whether to enable Search for any given chat turn, you can provide the tool explicitly.

In [9]:
nosearch_model = genai.GenerativeModel("gemini-1.5-flash-002")
chat = nosearch_model.start_chat()

# No search grounding.
r = chat.send_message("Hello friendly chatbot!")

# Enable search for just this turn.
r = chat.send_message(
    "Who took home the 2023 cricket world cup?", tools="google_search_retrieval"
)

Markdown(r.text)

Australia won the 2023 Cricket World Cup.  They defeated India in the final match by six wickets, securing their sixth World Cup title.


In [10]:
HTML(r.candidates[0].grounding_metadata.search_entry_point.rendered_content)

Or you can let the Gemini API calculate a likelihood that the response needs search backing, and define the threshold to use.

In [11]:
# The dynamic retrieval score is a probability, so the threshold
# must also be bound by [0, 1].
search_config = {
    "dynamic_retrieval_config": {"mode": "MODE_DYNAMIC", "dynamic_threshold": 0.5}
}

maybe_search_model = genai.GenerativeModel(
    "gemini-1.5-flash-002", tools={"google_search_retrieval": search_config}
)

chat = maybe_search_model.start_chat()

r = chat.send_message("Hello friendly chatbot!")
rc = r.candidates[0]
score = rc.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f"First turn: {score=}")

r = chat.send_message("Who took home the 2023 cricket world cup?")
rc = r.candidates[0]
score = rc.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f"Second turn: {score=}")
print()

display(Markdown(r.text))

First turn: score=0.400390625
Second turn: score=0.69140625



Australia won the 2023 Cricket World Cup.  They defeated India in the final match by six wickets, securing their sixth World Cup title.


In [12]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)